In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
from torchvision.transforms import ToTensor, Resize
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#yifan:Check if GPU is available
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('now is gpu')
else:
  device = torch.device("cpu")
  print('now is cpu')

now is gpu


In [ ]:
#load the json file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego/transforms_train.json') as f:
    data = json.load(f)

In [ ]:
transform_matrices = np.array([np.array(frame['transform_matrix']) for frame in data['frames']])

In [ ]:
transform_matrices.shape

(100, 4, 4)

In [ ]:
class subDataset():
  def __init__(self,train):
      self.train=train

  def __len__(self):
      return len(self.train)

  def __getitem__(self, index):
    data_transforms = transforms.Compose([
    transforms.ToTensor(),
    ])
    transform_data = data_transforms(self.train[index])
    return transform_data

In [ ]:
train_dataset = subDataset(transform_matrices)
batch_size = 8
dataloader_train = DataLoader(train_dataset, batch_size=batch_size)

In [ ]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super().__init__() #yifan:inherit the basic module
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=8, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(8),
            nn.ReLU() 
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(1),
            nn.LeakyReLU() #yifan:this is good for denosing since it has the range fomr -1 to 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
import torch.nn as nn
import torch.optim as optim
model = DenoisingAutoencoder().to(device) #yifan: initial the model use gpu
n_epochs = 10000
total_loss = []

# set to train mode
model.train()

# define the loss function
criterion = nn.MSELoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    train_loss = 0
    for data in dataloader_train:
        inputs = data
        noisy_inputs = inputs + 0.1 * torch.randn(inputs.shape) #yifan: 0 mean and 1 variance
        noisy_inputs = noisy_inputs.to(device)
        noisy_inputs = noisy_inputs.to(device)
        optimizer.zero_grad()
        noisy_inputs = noisy_inputs.float()
        outputs = model(noisy_inputs)
        inputs = inputs.float().to(device)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    total_loss.append(train_loss / len(dataloader_train))
    print('current Epoch is {} out of {}, current Loss is {}'.format(epoch+1, n_epochs, train_loss / len(dataloader_train)))
plt.plot(total_loss)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Loss Graph')
plt.show()
plt.save('loss.jpg')

In [ ]:
for data in transform_matrices:
  print(data)

In [ ]:
noised_data = transform_matrices +  0.1 * np.random.randn(*transform_matrices.shape)

In [ ]:
noised_data.shape

(100, 4, 4)

In [ ]:
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_corr/transforms_train.json', 'r') as infile:
    corr_data = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(corr_data['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = noised_data[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_corr/transforms_train.json', 'w') as outfile:
    json.dump(corr_data, outfile)

In [ ]:
model.eval()
noised_dataset = subDataset(noised_data)
batch_size = 8
noised_dataset = DataLoader(noised_dataset, batch_size=batch_size)

In [ ]:
#denoise_data_train=np.zeros(100)
denoise_data_train =[]
index = 0
with torch.no_grad():
    for data in noised_dataset:
        data = data.to(device)
        data = data.float()
        result = model(data).cpu().numpy()
        result = result.squeeze(1)
        #print(result.shape[0])
        for j in range(result.shape[0]):
          denoise_data_train.append(result[j])

In [ ]:
denoise_data_train = np.array(denoise_data_train)

In [ ]:
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_de/transforms_train.json', 'r') as infile:
    denoise_train = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(denoise_train['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = denoise_data_train[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_de/transforms_train.json', 'w') as outfile:
    json.dump(denoise_train, outfile)

<h2> for test

In [ ]:
#load the json file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego/transforms_test.json') as f:
    test = json.load(f)
transform_matrices_test = np.array([np.array(frame['transform_matrix']) for frame in test['frames']])

In [ ]:
test_dataset = subDataset(transform_matrices_test)
batch_size = 8
dataloader_test = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn
import torch.optim as optim
model = DenoisingAutoencoder().to(device) #yifan: initial the model use gpu
n_epochs = 10000
total_loss_test = []

# set to train mode
model.train()

# define the loss function
criterion = nn.MSELoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    train_loss = 0
    for data in dataloader_test:
        inputs = data
        noisy_inputs = inputs + 0.1 * torch.randn(inputs.shape) #yifan: 0 mean and 1 variance
        noisy_inputs = noisy_inputs.to(device)
        noisy_inputs = noisy_inputs.to(device)
        optimizer.zero_grad()
        noisy_inputs = noisy_inputs.float()
        outputs = model(noisy_inputs)
        inputs = inputs.float().to(device)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    total_loss_test.append(train_loss / len(dataloader_test))
    print('current Epoch is {} out of {}, current Loss is {}'.format(epoch+1, n_epochs, train_loss / len(dataloader_test)))

In [ ]:
noised_data_test = transform_matrices_test +  0.1 * np.random.randn(*transform_matrices_test.shape)
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_corr/transforms_test.json', 'r') as infile:
    corr_data_test = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(corr_data_test['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = noised_data_test[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_corr/transforms_test.json', 'w') as outfile:
    json.dump(corr_data_test, outfile)

In [ ]:
model.eval()
noised_dataset_test = subDataset(noised_data_test)
batch_size = 8
noised_dataset_test = DataLoader(noised_dataset_test, batch_size=batch_size)

In [ ]:
#denoise_data_train=np.zeros(100)
denoise_data_test =[]
index = 0
with torch.no_grad():
    for data in noised_dataset_test:
        data = data.to(device)
        data = data.float()
        result = model(data).cpu().numpy()
        result = result.squeeze(1)
        #print(result.shape[0])
        for j in range(result.shape[0]):
          denoise_data_test.append(result[j])

In [ ]:
denoise_data_test = np.array(denoise_data_test)

In [ ]:
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_de/transforms_test.json', 'r') as infile:
    denoise_test = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(denoise_test['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = denoise_data_test[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_de/transforms_test.json', 'w') as outfile:
    json.dump(denoise_test, outfile)

<h2> for val

In [ ]:
#load the json file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego/transforms_val.json') as f:
    val = json.load(f)
transform_matrices_val = np.array([np.array(frame['transform_matrix']) for frame in val['frames']])

In [ ]:
print(transform_matrices_val.shape)

(100, 4, 4)


In [ ]:
val_dataset = subDataset(transform_matrices_val)
batch_size = 8
dataloader_val = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn
import torch.optim as optim
model = DenoisingAutoencoder().to(device) #yifan: initial the model use gpu
n_epochs = 10000
total_loss_test = []

# set to train mode
model.train()

# define the loss function
criterion = nn.MSELoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    train_loss = 0
    for data in dataloader_val:
        inputs = data
        noisy_inputs = inputs + 0.1 * torch.randn(inputs.shape) #yifan: 0 mean and 1 variance
        noisy_inputs = noisy_inputs.to(device)
        noisy_inputs = noisy_inputs.to(device)
        optimizer.zero_grad()
        noisy_inputs = noisy_inputs.float()
        outputs = model(noisy_inputs)
        inputs = inputs.float().to(device)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    total_loss_test.append(train_loss / len(dataloader_test))
    print('current Epoch is {} out of {}, current Loss is {}'.format(epoch+1, n_epochs, train_loss / len(dataloader_test)))

In [ ]:
noised_data_val = transform_matrices_val +  0.1 * np.random.randn(*transform_matrices_val.shape)
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_corr/transforms_val.json', 'r') as infile:
    corr_data_val = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(corr_data_val['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = noised_data_val[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_corr/transforms_val.json', 'w') as outfile:
    json.dump(corr_data_val, outfile)

In [ ]:
model.eval()
noised_dataset_val = subDataset(noised_data_val)
batch_size = 8
noised_dataset_val = DataLoader(noised_dataset_val, batch_size=batch_size)

In [ ]:
#denoise_data_train=np.zeros(100)
denoise_data_val =[]
index = 0
with torch.no_grad():
    for data in noised_dataset_val:
        data = data.to(device)
        data = data.float()
        result = model(data).cpu().numpy()
        result = result.squeeze(1)
        #print(result.shape[0])
        for j in range(result.shape[0]):
          denoise_data_val.append(result[j])

In [ ]:
denoise_data_val = np.array(denoise_data_val)

In [ ]:
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_de/transforms_val.json', 'r') as infile:
    denoise_val = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(denoise_val['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = denoise_data_val[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_de/transforms_val.json', 'w') as outfile:
    json.dump(denoise_val, outfile)

<h2> another noise poisson

In [ ]:
#load the json file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego/transforms_train.json') as f:
    train = json.load(f)
transform_matrices_train = np.array([np.array(frame['transform_matrix']) for frame in train['frames']])

In [ ]:
train_dataset = subDataset(transform_matrices_train)
batch_size = 8
dataloader_train = DataLoader(train_dataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn
import torch.optim as optim
model = DenoisingAutoencoder().to(device) #yifan: initial the model use gpu
n_epochs = 10000
total_loss_train = []

# set to train mode
model.train()

# define the loss function
criterion = nn.MSELoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    train_loss = 0
    for data in dataloader_train:
        inputs = data
        noise = torch.tensor(np.random.poisson(2, inputs.shape))
        noisy_inputs = inputs + 0.1*noise #yifan: 0 mean and 1 variance
        # noisy_inputs = inputs + 0.1 * torch.randn(inputs.shape) #yifan: 0 mean and 1 variance
        noisy_inputs = noisy_inputs.to(device)
        optimizer.zero_grad()
        noisy_inputs = noisy_inputs.float()
        outputs = model(noisy_inputs)
        inputs = inputs.float().to(device)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    total_loss_train.append(train_loss / len(dataloader_train))
    print('current Epoch is {} out of {}, current Loss is {}'.format(epoch+1, n_epochs, train_loss / len(dataloader_train)))

In [ ]:
noised_data_train = transform_matrices_train +  0.1 * np.random.randn(*transform_matrices_train.shape)
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_corr/transforms_train.json', 'r') as infile:
    corr_data_train = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(corr_data_train['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = noised_data_train[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_corr/transforms_train.json', 'w') as outfile:
    json.dump(corr_data_train, outfile)

In [ ]:
model.eval()
noised_dataset_train = subDataset(noised_data_train)
batch_size = 8
noised_dataset_train = DataLoader(noised_dataset_train, batch_size=batch_size)

#denoise_data_train=np.zeros(100)
denoise_data_train =[]
index = 0
with torch.no_grad():
    for data in noised_dataset_train:
        data = data.to(device)
        data = data.float()
        result = model(data).cpu().numpy()
        result = result.squeeze(1)
        #print(result.shape[0])
        for j in range(result.shape[0]):
          denoise_data_train.append(result[j])
denoise_data_train = np.array(denoise_data_train)

In [ ]:
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_den/transforms_train.json', 'r') as infile:
    denoise_train = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(denoise_train['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = denoise_data_train[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_den/transforms_train.json', 'w') as outfile:
    json.dump(denoise_train, outfile)

<h2> possion for test

In [ ]:
#load the json file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego/transforms_test.json') as f:
    test = json.load(f)
transform_matrices_test = np.array([np.array(frame['transform_matrix']) for frame in test['frames']])

In [ ]:
test_dataset = subDataset(transform_matrices_test)
batch_size = 8
dataloader_test = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn
import torch.optim as optim
model = DenoisingAutoencoder().to(device) #yifan: initial the model use gpu
n_epochs = 10000
total_loss_test = []

# set to train mode
model.train()

# define the loss function
criterion = nn.MSELoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    train_loss = 0
    for data in dataloader_test:
        inputs = data
        noise = torch.tensor(np.random.poisson(2, inputs.shape))
        noisy_inputs = inputs + 0.1*noise #yifan: 0 mean and 1 variance
        # noisy_inputs = inputs + 0.1 * torch.randn(inputs.shape) #yifan: 0 mean and 1 variance
        noisy_inputs = noisy_inputs.to(device)
        optimizer.zero_grad()
        noisy_inputs = noisy_inputs.float()
        outputs = model(noisy_inputs)
        inputs = inputs.float().to(device)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    total_loss_train.append(train_loss / len(dataloader_test))
    print('current Epoch is {} out of {}, current Loss is {}'.format(epoch+1, n_epochs, train_loss / len(dataloader_test)))

In [ ]:
noised_data_test = transform_matrices_test +  0.1 * np.random.poisson(2,transform_matrices_test.shape) 
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_corr/transforms_test.json', 'r') as infile:
    corr_data_test = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(corr_data_test['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = noised_data_test[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_corr/transforms_test.json', 'w') as outfile:
    json.dump(corr_data_test, outfile)

In [ ]:
model.eval()
noised_dataset_test = subDataset(noised_data_test)
batch_size = 8
noised_dataset_test = DataLoader(noised_dataset_test, batch_size=batch_size)

#denoise_data_train=np.zeros(100)
denoise_data_test =[]
index = 0
with torch.no_grad():
    for data in noised_dataset_test:
        data = data.to(device)
        data = data.float()
        result = model(data).cpu().numpy()
        result = result.squeeze(1)
        #print(result.shape[0])
        for j in range(result.shape[0]):
          denoise_data_test.append(result[j])
denoise_data_test = np.array(denoise_data_test)

In [ ]:
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_den/transforms_test.json', 'r') as infile:
    denoise_test = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(denoise_test['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = denoise_data_test[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_den/transforms_test.json', 'w') as outfile:
    json.dump(denoise_test, outfile)

<h2> val

In [ ]:
#load the json file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego/transforms_val.json') as f:
    val = json.load(f)
transform_matrices_val = np.array([np.array(frame['transform_matrix']) for frame in val['frames']])

In [ ]:
val_dataset = subDataset(transform_matrices_val)
batch_size = 8
dataloader_val = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn
import torch.optim as optim
model = DenoisingAutoencoder().to(device) #yifan: initial the model use gpu
n_epochs = 10000
total_loss_test = []

# set to train mode
model.train()

# define the loss function
criterion = nn.MSELoss()

# define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    train_loss = 0
    for data in dataloader_val:
        inputs = data
        noise = torch.tensor(np.random.poisson(2, inputs.shape))
        noisy_inputs = inputs + 0.1*noise #yifan: 0 mean and 1 variance
        # noisy_inputs = inputs + 0.1 * torch.randn(inputs.shape) #yifan: 0 mean and 1 variance
        noisy_inputs = noisy_inputs.to(device)
        optimizer.zero_grad()
        noisy_inputs = noisy_inputs.float()
        outputs = model(noisy_inputs)
        inputs = inputs.float().to(device)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    total_loss_train.append(train_loss / len(dataloader_val))
    print('current Epoch is {} out of {}, current Loss is {}'.format(epoch+1, n_epochs, train_loss / len(dataloader_val)))

流式输出内容被截断，只能显示最后 5000 行内容。
current Epoch is 5001 out of 10000, current Loss is 0.1899797200010373
current Epoch is 5002 out of 10000, current Loss is 0.1908790274308278
current Epoch is 5003 out of 10000, current Loss is 0.1911541957121629
current Epoch is 5004 out of 10000, current Loss is 0.18888195919302794
current Epoch is 5005 out of 10000, current Loss is 0.19073386604969317
current Epoch is 5006 out of 10000, current Loss is 0.1894443487891784
current Epoch is 5007 out of 10000, current Loss is 0.18743026514465994
current Epoch is 5008 out of 10000, current Loss is 0.19028570961493713
current Epoch is 5009 out of 10000, current Loss is 0.18853280865229094
current Epoch is 5010 out of 10000, current Loss is 0.19157894242268342
current Epoch is 5011 out of 10000, current Loss is 0.19130652397871017
current Epoch is 5012 out of 10000, current Loss is 0.18922073107499343
current Epoch is 5013 out of 10000, current Loss is 0.1885464211496023
current Epoch is 5014 out of 10000, curren

In [ ]:
noised_data_val = transform_matrices_val +  0.1 * np.random.poisson(2,transform_matrices_val.shape) 
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_corr/transforms_val.json', 'r') as infile:
    corr_data_val = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(corr_data_val['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = noised_data_val[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_corr/transforms_val.json', 'w') as outfile:
    json.dump(corr_data_val, outfile)

In [ ]:
model.eval()
noised_dataset_val = subDataset(noised_data_val)
batch_size = 8
noised_dataset_val = DataLoader(noised_dataset_val, batch_size=batch_size)

#denoise_data_train=np.zeros(100)
denoise_data_val =[]
index = 0
with torch.no_grad():
    for data in noised_dataset_val:
        data = data.to(device)
        data = data.float()
        result = model(data).cpu().numpy()
        result = result.squeeze(1)
        #print(result.shape[0])
        for j in range(result.shape[0]):
          denoise_data_val.append(result[j])
denoise_data_val = np.array(denoise_data_val)

In [ ]:
# for the transofrm_train in lego_corr
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_den/transforms_val.json', 'r') as infile:
    denoise_val = json.load(infile)

# Assuming the 'frames' key contains the transform matrices in the JSON data
for i, frame in enumerate(denoise_val['frames']):
    # Update the values in the JSON data with the noisy transform_matrices
    frame['transform_matrix'] = denoise_data_val[i].tolist()

# Save the updated data back to the JSON file
with open('/content/drive/MyDrive/nerf_synthetic/nerf_synthetic/lego_noise2_den/transforms_val.json', 'w') as outfile:
    json.dump(denoise_val, outfile)